In [5]:
M=[{'city':'Dubai','temperature':33.},{'city':'London','temperature':12.},{'city':'Beijing','temperature':40.}]

from sklearn.feature_extraction import DictVectorizer

# 初始化特征抽取器
vec=DictVectorizer()

# 输出转化后的特征矩阵
print vec.fit_transform(M).toarray()

# 输出各维度特征的含义
print vec.get_feature_names()

[[ 0.  1.  0. 33.]
 [ 0.  0.  1. 12.]
 [ 1.  0.  0. 40.]]
['city=Beijing', 'city=Dubai', 'city=London', 'temperature']


由输出可知，特征向量化过程中。DictVectorizer对类别型和数值型特征的处理方式不同。

- 类别型(categorical)特征：借助原特征名称组合产生新特征，并用0/1二值方式进行量化
- 数值型(numerical)：维持原始特征值

**2.使用CountVectorizer且在不去掉停用词的条件下，对文本特征进行量化的朴素贝叶斯分类性能测试**

处理文本数据的方法：词袋法(Bag of Words)

- 不考虑词语出现的顺序，只将训练文本中的每个出现过的词汇单独视作一列特征；词表：不重复的词汇的集合；每条训练文本都可在高维度词表上映射出一个特征向量；
- 特征数值的常见计算方式：CountVectorizer & TfidfVectorizer
- CountVectorizer：只考虑每种词汇(Term)在该条训练文本中出现的频率(Term Frequency)
- TfidfVectorizer：既考量某一次会在当前文本中出现的频率，又考虑包含这个词汇的文本条数的倒数(Inverse Document Frequency),即训练的条目越多，TfidfVectorizer的特征量化就越有优势；可剔除在每条文本中都出现的常用词汇，以减少它们对模型分类决策的影响
- 停用词(Stop Words)：在每条文本中都出现的常用词汇，如the,a；停用词常在特征抽取中以黑名单的方式过滤掉，以提高模型的性能表现
- 区别：CountVectorizer只统计词频，而TfidfVectorizer还过滤掉了停用词

In [ ]:
from sklearn.datasets import fetch_20newsgroups
news=fetch_20newsgroups(subset='all')

from sklearn.cross_validation import train_test_split
X_train,y_train,X_test,y_test=train_test_split(news.data,news.target,test_size=0.25,random_state=33)

from sklearn.feature_extraction.text import CountVectorizer
count_vec=CountVectorizer()

# 只使用词频统计将原始训练和测试文本转化为特征向量
X_count_train=count_vec.fit_transform(X_train)
X_count_test=count_vec.transform(X_test)

# 导入naive bayes,默认配置初始化，使用CountVectorizer(未剔除停用词的)后的训练样本进行学习
from sklearn.naive_bayes import MultinomialNB
mnb_count=MultinomialNB()
mnb_count.fit(X_count_train,y_train)
y_count_predict=mnb_count.predict(X_test)

# 输出性能评估结果
from sklearn.metrics import classification_report
print 'Accuracy:',mnb_count.score(X_count_train,y_train)
print classification_report(y_test,y_count_predict,target_names=news.target_names)